In [4]:
#load modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import tensorflow as tf
import os
from PIL import Image
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
#load train data
train_dataset = []
with open(r"train.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    train_dataset.append(row)
#remove the header
train_dataset.pop(0)
    
#load test data
test_dataset = []
with open(r"test.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    test_dataset.append(row)
test_dataset.pop(0)
print() #just to make the output look nicer

In [6]:
train_labels = [i[0] for i in train_dataset]
train_images = [i[1:] for i in train_dataset]

In [7]:
#split data into training and validation sets
train_images, val_images, train_labels,val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)  

I just noticed that the integers were being cast as characters

In [19]:
#convert to numpy arrays
trainimages = [np.reshape(i, (28, 28)) for i in train_images]
trainimages = np.array(trainimages)
valimages = [np.reshape(i, (28, 28)) for i in val_images]
valimages = np.array(valimages)
testimages = [np.reshape(i, (28, 28)) for i in test_dataset]
testimages = np.array(testimages)

In [20]:
#convert chars to ints
trainimages = trainimages.astype(int)
valimages = valimages.astype(int)
testimages = testimages.astype(int)

In [30]:
#normalize pixel values
trainimages = trainimages/255.
testimages = testimages/255.

In [32]:
train_dataset = tf.data.Dataset.from_tensor_slices((trainimages, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((valimages, val_labels))

In [53]:
def create_model():
    model = tf.keras.Sequential()
    # model.add(tf.keras.Input(shape=(28,28)))#28x28 grayscale image
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    # model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # model.add(tf.keras.layers.Flatten())
    # model.add(tf.keras.layers.Dense(10,activation="softmax"))
    return model

In [54]:
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [55]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

Epoch 1/10


ValueError: in user code:

    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\keras\engine\